In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
data = pd.read_csv("last_data.csv")

In [3]:
data

,Unnamed: 0,ID,Dates,city,Hafta_Gunu,Racer,Racer_1,LAST_RACER_SAME_AS_CURRENT_RACER,LAST_RACER_SAME_AS_CURRENT_RACER_BINARY,Speed_in_Secs,...,K,DB,SK,SKG,GKR,YP,TGK,ÖG,SGKR,Winner
0,0,ENGİNYAR(12) KG SK,2015-08-07,Elazığ,Hafta İçi,S.KAPLAN,S.KAPLAN,True,1,157.51,...,0,0,1,0,0,0,0,0,0,0
1,1,TÜRK LOKUMU(8) KG,2015-07-29,Elazığ,Hafta İçi,R.YILDIRIM,R.YILDIRIM,True,1,167.86,...,0,0,0,0,0,0,0,0,0,0
2,2,KINGDOM(6) KG K,2015-07-29,Elazığ,Hafta İçi,C.TEPE,C.TEPE,True,1,147.87,...,1,0,0,0,0,0,0,0,0,0
3,3,ŞAHDİLDAR(10) KG K,2015-07-21,Ankara,Hafta İçi,MÜS.ÇELİK,MÜS.ÇELİK,True,1,74.67,...,1,0,0,0,0,0,0,0,0,0
4,4,DEHERE VERE(8) SKG SK,2015-07-21,Ankara,Hafta İçi,E.METİN,E.METİN,True,1,73.96,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12354,12354,ADAMINDİBİ(1) KG SK,2019-12-30,Bursa,Hafta İçi,E.ERGÜR,F.ARSLAN,False,0,110.45,...,0,0,1,0,0,0,0,0,0,0
12355,12355,KUBİ DAYI(10) KG DB SK,2019-12-29,Adana,Hafta Sonu,T.BALİ,M.ÖZDEMİR,False,0,131.47,...,0,1,1,0,0,0,0,0,0,0
12356,12356,KING OF DIAMONDS(13) DB SK,2019-12-30,Şanlıurfa,Hafta İçi,R.DOĞAN,R.DOĞAN,True,1,150.31,...,0,1,1,0,0,0,0,0,0,0
12357,12357,BEY POYRAZ(2) SK,2019-12-30,Şanlıurfa,Hafta İçi,İ.AKYAVUZ,İ.AKYAVUZ,True,1,102.02,...,0,0,1,0,0,0,0,0,0,0


In [4]:
categorical_columns = list(data.loc[:, data.dtypes == np.object].columns)

In [5]:
categorical_columns

['ID',
 'Dates',
 'city',
 'Hafta_Gunu',
 'Racer',
 'Racer_1',
 'kosu_saati',
 'at_ana',
 'at_baba',
 'at_soyu',
 'Mevsim']

In [6]:
# kategorik değişkenler df'den çıkarılıyor.

df = data.drop(categorical_columns, axis=1)

In [7]:
# boolean ve önceden normalize edilmiş çıkarılması gereken değişkenler çıkarılıyor.

df = df.drop(["LAST_RACER_SAME_AS_CURRENT_RACER", "Unnamed: 0", 
              "LAST_RACER_SAME_AS_CURRENT_RACER_BINARY", "RANK", "Ranking_Score_Normalized.1", 
             "RANK_LATEST_1ST", "AG_RANKING_Score_Normalized"], axis=1)

In [8]:
df = df.iloc[:,:-13]

In [9]:
# neural network regressor eğitimi için hazır olan dataframe

df

,Speed_in_Secs,GANYAN_TRANS,GANYAN_TRANS_INT,AG_Percentage,AG_RANK_AMONG_GROUP,Ranking_Score_Normalized,First_Position_Percentage,Second_Position_Percentage,Third_Position_Percentage,Fourth_Position_Percentage,...,BIRINCILIK_PERCENTAGE,IKINCILIK_PERCENTAGE,UCUNCULUK_PERCENTAGE,DORDUNCULUK_PERCENTAGE,SPEED_LATEST_1ST,SPEED_LATEST_2nd,SPEED_LATEST_3rd,SPEED_LATEST_4th,SPEED_LATEST_5th,RANK.1
0,157.51,12.65,1265,2.0,11,46,8,8,7,9,...,2,8,0,17,11.4278,10.8861,11.4640,10.9800,9.3825,8
1,167.86,15.85,1585,1.0,12,7,4,6,6,7,...,4,4,9,18,10.1275,13.8776,13.4525,13.5883,13.9714,13
2,147.87,10.10,1010,3.0,11,53,8,8,7,8,...,10,10,16,8,13.5254,14.8968,13.9792,14.0046,13.7287,7
3,74.67,3.55,355,16.0,2,39,14,13,13,12,...,0,0,0,0,16.0707,0.0000,0.0000,0.0000,0.0000,7
4,73.96,8.80,880,5.0,8,80,8,9,10,10,...,0,0,100,0,16.2250,0.0000,0.0000,0.0000,0.0000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12354,110.45,43.80,4380,0.0,0,53,0,0,0,0,...,0,0,0,0,0.0000,12.6754,12.2897,13.7662,0.0000,8
12355,131.47,37.00,3700,1.0,10,54,2,2,6,6,...,7,12,4,9,0.0000,0.0000,15.2126,0.0000,0.0000,6
12356,150.31,7.95,795,2.0,5,26,8,9,10,10,...,17,5,17,14,13.3058,13.7651,14.7230,14.7424,15.0539,12
12357,102.02,15.70,1570,0.0,0,40,12,10,10,9,...,0,0,0,50,13.7228,12.9766,14.1961,15.1649,14.9421,10


In [10]:
from sklearn.neural_network import MLPRegressor

In [11]:
# RANK.1 hedef değişken olarak belirlenip geriye kalanlar bağımsız değişken olarak model için hazırlanıyor.

X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42)

In [13]:
# ilk neural network regressor eğitimi yapılıyor.

mlp_regr = MLPRegressor(alpha=0.02, hidden_layer_sizes=(15, 10, 5)).fit(X_train, y_train)

In [14]:
y_pred = mlp_regr.predict(X_test)

In [15]:
r2_score(y_test, y_pred)

0.7645603117358951

In [16]:
np.sqrt(mean_squared_error(y_test, y_pred))

1.76638584913819

In [17]:
d = pd.DataFrame(y_pred)
d.columns=["y_pred"]

In [18]:
y_pred

array([12.09232274, 11.07921619,  8.62609368, ...,  9.24165085,
        7.91365455, 13.67731754])

In [19]:
yt = np.array(y_test["RANK.1"])
yt

array([12, 13, 10, ..., 11,  7, 10])

In [20]:
# prediction ve reality olarak y_test ve y_pred değerleri karşılaştırılıyor. ve Dataframe haline getiriliyor.

dff = pd.DataFrame(y_pred)
dff.columns = ["predictions"]
dff["reality"] = np.array(y_test["RANK.1"])

In [21]:
# üst sayıya yuvarlanarak tahmin ediliyor
# 15.76 = 16,   13.25 = 13 gibi

# her eşleşen değer ile count 1 arttırılıyor sonucunda y_test ve y_pred eşleşen değerler toplam değerlere bölünerek
# doğruluk yüzdesi belirleniyor.

count=0
for i in range(0,len(dff["reality"])):
#     print(round(dff["predictions"][i],0))
    if int(round(dff["predictions"][i],0)) == int(dff["reality"][i]):
        count = count + 1
print(count)

624


In [22]:
# tahmin yüzdesi
595/2472

0.2406957928802589

In [23]:
# üst sayıya yuvarlanmadan direkt int'e çevrilip eşleniyor
# 13.25 = 13,   15,98 = 15 gibi.

count=0
for i in range(0,len(dff["reality"])):
#     print(round(dff["predictions"][i],0))
    if int(dff["predictions"][i]) == int(dff["reality"][i]):
        count = count + 1
print(count)

671


In [24]:
# tahmin yüzdesi
669/2472

0.27063106796116504

## Model Tuning

In [25]:
mlp_param = {"alpha": [0.1,0.01,0.02,0.005],
             "hidden_layer_sizes": [(12,8,4), (15,10,5), (7,7,7)]}

In [27]:
mlp_cv_model = GridSearchCV(mlp_regr, mlp_param, cv=10, n_jobs=-1, verbose=2)

In [28]:
mlp_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.2min finished


GridSearchCV(cv=10,
             estimator=MLPRegressor(alpha=0.02, hidden_layer_sizes=(15, 10, 5)),
             n_jobs=-1,
             param_grid={'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(12, 8, 4), (15, 10, 5),
                                                (7, 7, 7)]},
             verbose=2)

In [29]:
mlp_cv_model.best_params_

{'alpha': 0.1, 'hidden_layer_sizes': (7, 7, 7)}

In [30]:
mlp_tuned = MLPRegressor(alpha=0.1, hidden_layer_sizes=(7, 7, 7)).fit(X_train, y_train)

In [31]:
y_pred_tuned = mlp_tuned.predict(X_test)

In [32]:
r2_score(y_test, y_pred_tuned)

0.8371519877353418

In [33]:
# tune edilmiş yeni değerler ile tekrar tahmin edilen ve gerçek değerler karşılaştırılıyor.

dff2 = pd.DataFrame(y_pred_tuned)
dff2.columns = ["predictions"]
dff2["reality"] = np.array(y_test["RANK.1"])

In [40]:
# üst sayıya yuvarlanarak tahmin ediliyor

count=0
for i in range(0,len(dff2["reality"])):
#     print(round(dff["predictions"][i],0))
    if int(round(dff2["predictions"][i],0)) == int(dff2["reality"][i]):
        count = count + 1
print(count/len(y_pred_tuned))
# count = 767 / 2742

0.31027508090614886


In [37]:
# üst sayıya yuvarlanmadan direkt int'e çevrilip eşleniyor

count=0
for i in range(0,len(dff2["reality"])):
#     print(round(dff["predictions"][i],0))
    if int(dff2["predictions"][i]) == int(dff2["reality"][i]):
        count = count + 1
print(count/len(y_pred_tuned))

0.2698220064724919


In [42]:
dff2

,predictions,reality
0,11.008557,12
1,10.800574,13
2,9.421778,10
3,14.436241,16
4,4.510502,4
...,...,...
2467,5.066948,6
2468,11.981612,15
2469,9.347790,11
2470,7.163480,7


#### Model Tuning 2

In [58]:
mlp_param2 = {"alpha": [0.1,0.01,0.001,0.2,0.4,0.8,1],
             "hidden_layer_sizes": [(11,11,11), (15,15,15), (20,20,20)]}

In [59]:
mlp_cv_model2 = GridSearchCV(mlp_tuned, mlp_param2, cv=10, n_jobs=-1, verbose=2)

In [60]:
mlp_cv_model2.fit(X_train, y_train)

Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:  9.2min finished


GridSearchCV(cv=10,
             estimator=MLPRegressor(alpha=0.1, hidden_layer_sizes=(7, 7, 7)),
             n_jobs=-1,
             param_grid={'alpha': [0.1, 0.01, 0.001, 0.2, 0.4, 0.8, 1],
                         'hidden_layer_sizes': [(11, 11, 11), (15, 15, 15),
                                                (20, 20, 20)]},
             verbose=2)

In [61]:
mlp_cv_model2.best_params_

{'alpha': 0.8, 'hidden_layer_sizes': (11, 11, 11)}

In [95]:
mlp_tuned2 = MLPRegressor(alpha=4, hidden_layer_sizes=(5, 4, 3)).fit(X_train, y_train)

In [96]:
y_pred_tuned2 = mlp_tuned2.predict(X_test)

In [97]:
r2_score(y_test, y_pred_tuned2)

0.8397726623018105

In [136]:
# random_state değer atandı her uygulama sonrası farklı r2_score dönüyordu.

mlp_tuned2 = MLPRegressor(alpha=8, hidden_layer_sizes=(6, 5, 4), random_state=10).fit(X_train, y_train)
y_pred_tuned2 = mlp_tuned2.predict(X_test)
r2_score(y_test, y_pred_tuned2)

0.8244650850308797

In [180]:
mlp = MLPRegressor(alpha=8, hidden_layer_sizes=(6, 5, 4, 4), verbose=True, 
                    learning_rate='adaptive', max_iter=500, random_state=1)

In [181]:
mlp.fit(X_train, y_train)

Iteration 1, loss = 78588.22987095
Iteration 2, loss = 12787.36437695
Iteration 3, loss = 4656.81993350
Iteration 4, loss = 2762.51005484
Iteration 5, loss = 1835.71249967
Iteration 6, loss = 1239.32959268
Iteration 7, loss = 838.12759485
Iteration 8, loss = 566.65479911
Iteration 9, loss = 380.92918727
Iteration 10, loss = 253.09827614
Iteration 11, loss = 168.41093112
Iteration 12, loss = 111.49474600
Iteration 13, loss = 73.86188841
Iteration 14, loss = 49.42767991
Iteration 15, loss = 33.62979379
Iteration 16, loss = 23.43600111
Iteration 17, loss = 17.04595786
Iteration 18, loss = 13.02610835
Iteration 19, loss = 10.52377314
Iteration 20, loss = 6.47465518
Iteration 21, loss = 5.76946486
Iteration 22, loss = 5.49323891
Iteration 23, loss = 5.23769981
Iteration 24, loss = 5.04767726
Iteration 25, loss = 4.79788456
Iteration 26, loss = 4.54141314
Iteration 27, loss = 4.31697878
Iteration 28, loss = 4.03350352
Iteration 29, loss = 3.75591881
Iteration 30, loss = 3.51290542
Iteration 

Iteration 252, loss = 1.47568080
Iteration 253, loss = 1.52127736
Iteration 254, loss = 1.46622287
Iteration 255, loss = 1.46432033
Iteration 256, loss = 1.52238492
Iteration 257, loss = 1.48540322
Iteration 258, loss = 1.51427342
Iteration 259, loss = 1.47821733
Iteration 260, loss = 1.48882705
Iteration 261, loss = 1.45106600
Iteration 262, loss = 1.50458753
Iteration 263, loss = 1.44888370
Iteration 264, loss = 1.48480917
Iteration 265, loss = 1.44140966
Iteration 266, loss = 1.46219104
Iteration 267, loss = 1.44859316
Iteration 268, loss = 1.45623941
Iteration 269, loss = 1.48437964
Iteration 270, loss = 1.44825051
Iteration 271, loss = 1.58712138
Iteration 272, loss = 1.49451007
Iteration 273, loss = 1.46061461
Iteration 274, loss = 1.43926919
Iteration 275, loss = 1.47146426
Iteration 276, loss = 1.62983511
Iteration 277, loss = 1.47026415
Iteration 278, loss = 1.43760701
Iteration 279, loss = 1.48231552
Iteration 280, loss = 1.45622316
Iteration 281, loss = 1.43293162
Iteration 

MLPRegressor(alpha=8, hidden_layer_sizes=(6, 5, 4, 4), learning_rate='adaptive',
             max_iter=500, random_state=1, verbose=True)

In [182]:
y22 = mlp.predict(X_test)

In [183]:
r2_score(y_test, y22)

0.8491347542931635

In [184]:
dff3 = pd.DataFrame(y22)
dff3.columns = ["predictions"]
dff3["reality"] = np.array(y_test["RANK.1"])

In [189]:
# üst sayıya yuvarlanarak tahmin ediliyor

count=0
for i in range(0,len(dff3["reality"])):

    if int(round(dff3["predictions"][i],0)) == int(dff3["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y22)*100,2))


doğru tahmin yüzdesi:  31.19


In [190]:
# yuvarlama olmadan sayılar hesaplanıyor

count=0
for i in range(0,len(dff3["reality"])):
#     print(round(dff["predictions"][i],0))
    if int((dff3["predictions"][i])) == int(dff3["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y22)*100,2))

doğru tahmin yüzdesi:  31.55


In [199]:
mlp_2 = MLPRegressor(alpha=2, hidden_layer_sizes=(12, 12, 12, 12), verbose=True, 
                    learning_rate='adaptive', max_iter=500, random_state=2)

In [200]:
mlp_2.fit(X_train, y_train)

Iteration 1, loss = 26.38864718
Iteration 2, loss = 11.90880722
Iteration 3, loss = 7.63251823
Iteration 4, loss = 5.85384416
Iteration 5, loss = 4.29842484
Iteration 6, loss = 3.31201518
Iteration 7, loss = 2.71713638
Iteration 8, loss = 2.42411851
Iteration 9, loss = 2.39165278
Iteration 10, loss = 2.35180091
Iteration 11, loss = 2.21545215
Iteration 12, loss = 2.18921079
Iteration 13, loss = 2.15269974
Iteration 14, loss = 2.12491938
Iteration 15, loss = 2.11186354
Iteration 16, loss = 2.09160923
Iteration 17, loss = 2.05606734
Iteration 18, loss = 2.01478714
Iteration 19, loss = 2.05220449
Iteration 20, loss = 1.96923797
Iteration 21, loss = 1.94511071
Iteration 22, loss = 1.91178165
Iteration 23, loss = 1.87246217
Iteration 24, loss = 1.81977117
Iteration 25, loss = 1.78776652
Iteration 26, loss = 1.76994545
Iteration 27, loss = 1.76715878
Iteration 28, loss = 1.75721868
Iteration 29, loss = 1.71317319
Iteration 30, loss = 1.66871514
Iteration 31, loss = 1.65076577
Iteration 32, l

MLPRegressor(alpha=2, hidden_layer_sizes=(12, 12, 12, 12),
             learning_rate='adaptive', max_iter=500, random_state=2,
             verbose=True)

In [201]:
y2 = mlp_2.predict(X_test)

In [202]:
r2_score(y_test, y2)

0.8558513286496768

In [215]:
mlp = MLPRegressor(alpha=8, hidden_layer_sizes=(6, 5, 5, 3), verbose=True, 
                    learning_rate='adaptive', max_iter=500, random_state=4)

In [216]:
mlp.fit(X_train, y_train)

Iteration 1, loss = 14.20816826
Iteration 2, loss = 9.14093373
Iteration 3, loss = 6.88717367
Iteration 4, loss = 5.56690937
Iteration 5, loss = 4.72112710
Iteration 6, loss = 4.06739507
Iteration 7, loss = 3.67460060
Iteration 8, loss = 3.30981917
Iteration 9, loss = 3.08505445
Iteration 10, loss = 2.92156129
Iteration 11, loss = 2.80205108
Iteration 12, loss = 2.72431371
Iteration 13, loss = 2.72647896
Iteration 14, loss = 2.63838971
Iteration 15, loss = 2.52916897
Iteration 16, loss = 2.51344902
Iteration 17, loss = 2.43962575
Iteration 18, loss = 2.41732590
Iteration 19, loss = 2.37907144
Iteration 20, loss = 2.33202695
Iteration 21, loss = 2.32156711
Iteration 22, loss = 2.25078768
Iteration 23, loss = 2.24202764
Iteration 24, loss = 2.26540099
Iteration 25, loss = 2.24395287
Iteration 26, loss = 2.17241784
Iteration 27, loss = 2.14052631
Iteration 28, loss = 2.10340592
Iteration 29, loss = 2.12852637
Iteration 30, loss = 2.18657387
Iteration 31, loss = 2.11240980
Iteration 32, lo

MLPRegressor(alpha=8, hidden_layer_sizes=(6, 5, 5, 3), learning_rate='adaptive',
             max_iter=500, random_state=4, verbose=True)

In [229]:
yhat = mlp.predict(X_test)
r2_score(y_test, yhat)

0.8195003566536093

In [231]:
# tahminleri yhat'i int değere çevirerek yapıyoruz

yhat = mlp.predict(X_test)
r2_score(y_test, yhat.astype('int32') )

0.7867196733535202

In [230]:
yhat

array([11.37017494, 10.37390739,  8.61416813, ...,  8.97391199,
        7.64230629, 10.86070656])

In [226]:
yhat.astype('int32') 

array([11, 10,  8, ...,  8,  7, 10], dtype=int32)

In [219]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, yhat)

0.8202962590963786

### predictions ve reality sonuçları arasında LinearRegression ile denklem kurulup daha doğru sonuçlara ulaşılmaya çalışılıyor

In [220]:
dff3

,predictions,reality
0,11.159000,12
1,11.145561,13
2,9.152412,10
3,15.425638,16
4,4.967270,4
...,...,...
2467,6.201878,6
2468,12.324198,15
2469,9.512863,11
2470,7.405432,7


In [239]:
xx = dff3["predictions"]
yy = dff3["reality"]

In [241]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

xx = sm.add_constant(xx) 

reg = LinearRegression().fit(xx, yy)
reg.coef_

array([0.        , 1.01539373])

In [243]:
reg.intercept_

-0.22892042755602837

In [244]:
def fonk(x):
    return x*1.01539373 -0.22892042755602837

In [246]:
fonk(12.324198)

12.284992948922513

## predictions ve reality ile OLS

In [21]:
dff

,predictions,reality
0,12.092323,12
1,11.079216,13
2,8.626094,10
3,16.328157,16
4,5.438222,4
...,...,...
2467,5.405540,6
2468,11.970637,15
2469,9.241651,11
2470,7.913655,7


In [22]:
import statsmodels.api as sm
X = dff[["predictions"]]

X = sm.add_constant(X)  

y= dff["reality"]

In [23]:
lm = sm.OLS(y, X)
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                reality   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.786
Method:                 Least Squares   F-statistic:                     9083.
Date:                Mon, 28 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:31:43   Log-Likelihood:                -4794.8
No. Observations:                2472   AIC:                             9594.
Df Residuals:                    2470   BIC:                             9605.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.5221      0.096      5.434      0.000       0.334       0.710
predictions     0.9023      0.009     95.307      0.000       0.884       0.921
==============================================================================
Omnibus:                       38.970   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               57.858
Skew:                           0.165   Prob(JB):                     2.73e-13
Kurtosis:                       3.673   Cond. No.                         29.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# katsayılar ile bulunan denklemin fonksiyonu 

def fonk(x):
    return x*0.9023 + 0.5221

In [29]:
fonk(11.970637)

11.323205765099999

In [33]:
dff[dff["predictions"]-dff["reality"]>1]

,predictions,reality
4,5.438222,4
6,13.174056,12
7,11.196901,10
12,10.524541,8
14,11.038665,9
...,...,...
2456,7.971881,6
2457,11.636884,10
2458,12.884202,10
2461,9.280015,8


In [37]:
# ols sonucunda bulunan denklemdeki değerlere predictions değerleri yazılıyor. ve tahminler güncelleniyor.
# güncellenen değerler ols kolonuna yazılıyor

dff["ols"] = dff["predictions"].apply(lambda x: fonk(x))

In [45]:
dff

,predictions,reality,ols
0,12.092323,12,11.433003
1,11.079216,13,10.518877
2,8.626094,10,8.305424
3,16.328157,16,15.254996
4,5.438222,4,5.429008
...,...,...,...
2467,5.405540,6,5.399519
2468,11.970637,15,11.323206
2469,9.241651,11,8.860842
2470,7.913655,7,7.662590


In [42]:
count=0
for i in range(0,len(dff["reality"])):
#     print(round(dff["predictions"][i],0))
    if int((dff["predictions"][i])) == int(dff["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y)*100,2))

doğru tahmin yüzdesi:  25.49


In [41]:
# yuvarlama olmadan sayılar hesaplanıyor

count=0
for i in range(0,len(dff["reality"])):
#     print(round(dff["predictions"][i],0))
    if int((dff["ols"][i])) == int(dff["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y)*100,2))

doğru tahmin yüzdesi:  24.23


In [43]:

count=0
for i in range(0,len(dff["reality"])):

    if int(round(dff["predictions"][i],0)) == int(dff["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y)*100,2))

doğru tahmin yüzdesi:  22.69


In [44]:
count=0
for i in range(0,len(dff["reality"])):

    if int(round(dff["ols"][i],0)) == int(dff["reality"][i]):
        count = count + 1
print("doğru tahmin yüzdesi: ",round(count/len(y)*100,2))

doğru tahmin yüzdesi:  23.91
